# Convert  Object Store datapoints - Airbus NationWide - into Visual search Map

In [1]:
import pandas
print(pandas.__version__)

0.24.2


In [2]:
import folium
print(folium.__version__)

0.14.0


In [3]:
import numpy as np 
import pandas as pd 
import folium
from folium.plugins import Search
from functools import reduce
import math
# Convert BNG to LAT/LON
from bng_latlon import OSGB36toWGS84
input_coord_csv_file='data/output/COORD_LATLON_NBS_Portfolio_Bristol_230216_csv/file_COORD_LATLON_NBS_Portfolio_Bristol_230216_csv_v2.csv'
df_nationwide_loanbook = pd.read_csv(input_coord_csv_file, header = 0 )


In [4]:
df_nationwide_loanbook.describe()

,ID,UPRN,POSTCODE_FLAG,EASTING,NORTHING,RHIGHSCORE,RHIGHSCORE_ADJ,CHIGHSCORE,CHIGHSCORE_ADJ,SWHIGHSCORE,...,RCP45_2050S_DEF_C_RR,RCP85_2050S_UNDEF_F_RR,RCP85_2050S_UNDEF_P_RR,RCP85_2050S_UNDEF_T_RR,RCP85_2050S_UNDEF_C_RR,RCP85_2050S_DEF_F_RR,RCP85_2050S_DEF_T_RR,RCP85_2050S_DEF_C_RR,LAT,LON
count,18085.000000,1.808500e+04,18085.000000,18085.000000,18085.000000,18085.000000,18085.00000,18085.000000,18085.00000,18085.000000,...,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18075.000000,18085.000000,18085.000000
mean,9043.000000,8.901320e+09,0.002765,363001.261979,174118.554777,0.517556,0.44031,0.370805,0.09953,0.551562,...,4.389986,1.399336,3.404869,1.235076,5.336985,1.399336,0.891286,5.030152,51.464793,-2.534007
std,5220.834145,2.814949e+10,0.052509,5482.263478,5092.701827,2.950690,2.63575,2.569122,1.14896,1.643107,...,15.618402,10.035297,13.124174,10.103181,18.234758,10.035297,7.390518,17.074913,0.045734,0.078985
min,1.000000,1.470000e+02,0.000000,350002.000000,160034.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.338311,-2.721264
25%,4522.000000,1.116180e+05,0.000000,359164.950000,170934.000000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.436014,-2.589200
50%,9043.000000,3.731460e+05,0.000000,361955.000000,174209.250000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.465526,-2.549159
75%,13564.000000,5.924060e+05,0.000000,365945.000000,177516.230000,0.000000,0.00000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.495231,-2.491612
max,18085.000000,2.000030e+11,1.000000,379995.000000,186975.000000,22.000000,22.00000,20.000000,20.00000,10.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,51.581106,-2.288628


In [5]:
df_nationwide_loanbook.dtypes

ID                          int64
UPRN                      float64
POSTCODE_FLAG               int64
EASTING                   float64
NORTHING                  float64
RHIGHSCORE                  int64
RHIGHSCORE_ADJ              int64
CHIGHSCORE                  int64
CHIGHSCORE_ADJ              int64
SWHIGHSCORE                 int64
SWHIGHSCORE_ADJ             int64
GWHIGHSCORE                 int64
COMBINEDSCORE               int64
COMBINEDSCORE_ADJ           int64
R_RCP26_2050_UD           float64
R_RCP26_2050_D            float64
S_RCP26_2050_UD           float64
S_RCP26_2050_D            float64
C_RCP26_2050_UD           float64
C_RCP26_2050_D            float64
COMB_RCP26_2050_UD        float64
COMB_RCP26_2050_D         float64
R_RCP45_2050_UD           float64
R_RCP45_2050_D            float64
S_RCP45_2050_UD           float64
S_RCP45_2050_D            float64
C_RCP45_2050_UD           float64
C_RCP45_2050_D            float64
COMB_RCP45_2050_UD        float64
COMB_RCP45_205

In [6]:
df_nationwide_loanbook.head(2)

,ID,UPRN,POSTCODE_FLAG,EASTING,NORTHING,RHIGHSCORE,RHIGHSCORE_ADJ,CHIGHSCORE,CHIGHSCORE_ADJ,SWHIGHSCORE,...,RCP85_2050S_UNDEF_F_RR,RCP85_2050S_UNDEF_P_RR,RCP85_2050S_UNDEF_T_RR,RCP85_2050S_UNDEF_C_RR,RCP85_2050S_DEF_F_RR,RCP85_2050S_DEF_T_RR,RCP85_2050S_DEF_C_RR,CORD,LAT,LON
0,1,147.0,0,358754.72,179718.88,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(51.514878, -2.595774)",51.514878,-2.595774
1,2,148.0,0,358946.00,179716.00,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(51.514866, -2.593018)",51.514866,-2.593018


In [7]:
df_nationwide_loanbook.tail(3)

,ID,UPRN,POSTCODE_FLAG,EASTING,NORTHING,RHIGHSCORE,RHIGHSCORE_ADJ,CHIGHSCORE,CHIGHSCORE_ADJ,SWHIGHSCORE,...,RCP85_2050S_UNDEF_F_RR,RCP85_2050S_UNDEF_P_RR,RCP85_2050S_UNDEF_T_RR,RCP85_2050S_UNDEF_C_RR,RCP85_2050S_DEF_F_RR,RCP85_2050S_DEF_T_RR,RCP85_2050S_DEF_C_RR,CORD,LAT,LON
18082,18085,2.000030e+11,0,373226.21,165042.78,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(51.383794, -2.386118)",51.383794,-2.386118
18083,18083,2.000030e+11,0,361835.00,166101.00,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(51.392655, -2.549913)",51.392655,-2.549913
18084,18084,2.000030e+11,0,355767.00,162180.00,0,0,0,0,5,...,0.0,3.0,0.0,3.0,0.0,0.0,3.0,"(51.356959, -2.636631)",51.356959,-2.636631


In [8]:
# CONTINUE HERE LATER

In [9]:
# Increase display in Jupyter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
df_nationwide_loanbook.dtypes

ID                          int64
UPRN                      float64
POSTCODE_FLAG               int64
EASTING                   float64
NORTHING                  float64
RHIGHSCORE                  int64
RHIGHSCORE_ADJ              int64
CHIGHSCORE                  int64
CHIGHSCORE_ADJ              int64
SWHIGHSCORE                 int64
SWHIGHSCORE_ADJ             int64
GWHIGHSCORE                 int64
COMBINEDSCORE               int64
COMBINEDSCORE_ADJ           int64
R_RCP26_2050_UD           float64
R_RCP26_2050_D            float64
S_RCP26_2050_UD           float64
S_RCP26_2050_D            float64
C_RCP26_2050_UD           float64
C_RCP26_2050_D            float64
COMB_RCP26_2050_UD        float64
COMB_RCP26_2050_D         float64
R_RCP45_2050_UD           float64
R_RCP45_2050_D            float64
S_RCP45_2050_UD           float64
S_RCP45_2050_D            float64
C_RCP45_2050_UD           float64
C_RCP45_2050_D            float64
COMB_RCP45_2050_UD        float64
COMB_RCP45_205

In [11]:
df_nationwide_loanbook.shape

(18085, 201)

In [12]:
lat= 51.26969
lang= -2.46943

In [13]:
#Color codes for EPC rating scale
colors_epc={
    '':'#deebf6',
    'A':'#00a54f',
    'B':'#4cb848',
    'C':'#bed630',
    'D':'#fff101',
    'E':'#fcb814',
    'F':'#f36e21',
    'G':'#ee1d23',
}

In [14]:
#Color codes for flood rating scale
colors_flood={
    '':'#deebf6',
    '0':'#deebf6',
    0:'#deebf6',
    'No Colour':'#deebf6',
    'green':'#a5d45d',
    'amber':'#ecc63a',
    'red':'#d92424',
    'black 1':'#919191',
    'black 2':'#242424',
}

In [15]:
# Creating the popup labels for energy rating data

def popup_html_epc(row):
    i = row
    uprn_id= df_nationwide_loanbook['UPRN'].iloc[i] 
    Address= ' MASKED ADDRESS '+ str( df_nationwide_loanbook['POSTCODE_FLAG'].iloc[i] ) +'         '+ str( df_nationwide_loanbook['EASTING'].iloc[i] ) +' '+ str( df_nationwide_loanbook['NORTHING'].iloc[i] ) +'         '+ str( df_nationwide_loanbook['LAT'].iloc[i] ) +' '+ str( df_nationwide_loanbook['LON'].iloc[i] )
    Epc_Rating = df_nationwide_loanbook['CURRENT_ENERGY_RATING'].iloc[i]
    Epc_Energy_Cons_Kwh = df_nationwide_loanbook['ENERGY_CONSUMPTION_CURRENT'].iloc[i]
    Co2_Emission = df_nationwide_loanbook['CO2_EMISSIONS_CURRENT'].iloc[i]

    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">UPRN ID:{}</h4>""".format(uprn_id) + """</center>
<center> <table style="height: 126px; width: 305px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Address </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">"""+ Address + """</td>
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Epc Rating </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Epc_Rating) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Epc Energy Consumption Kwh </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{} kWh/m\u00b2/year</td>""".format(Epc_Energy_Cons_Kwh) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">CO\u2082 Emission Current</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Co2_Emission) + """
</tr>
</tbody>
</table></center>
</html>
"""
    return html

In [16]:
# Creating the popup labels for flood rating data

def popup_html_flood(row):
    i = row
    uprn_id=df_nationwide_loanbook['UPRN'].iloc[i] 
    address= ' MASKED ADDRESS '+ str( df_nationwide_loanbook['POSTCODE_FLAG'].iloc[i] ) +'         '+ str( df_nationwide_loanbook['EASTING'].iloc[i] ) +' '+ str( df_nationwide_loanbook['NORTHING'].iloc[i] ) +'         '+ str( df_nationwide_loanbook['LAT'].iloc[i] ) +' '+ str( df_nationwide_loanbook['LON'].iloc[i] )
    river_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    coastal_flooding_first = df_nationwide_loanbook['RHIGHSCORE_ADJ'].iloc[i]
    surf_water_flooding_first = df_nationwide_loanbook['CHIGHSCORE'].iloc[i]
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">UPRN ID:{}</h4>""".format(uprn_id) + """</center>
<center> <table style="height: 130px; width: 500px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Address </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Return Period at which River Flooding First Occurred </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(river_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Return Period at which Coastal Flooding First Occurred  </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(coastal_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Return Period at which River Surface Water Flooding First Occurred  </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(surf_water_flooding_first) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Combined Score - Undefended River, Coastal and Surface Water</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
</tbody>
</table>
</center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">River</h5>
<center> <table style="height: 130px; width: 500px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">20 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">75 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">100 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">200 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">1000 years </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Max. Depth of Flooding in the River Undefended in Meters </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">River Undefended Flood Score </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property Modelled for Undefended River Flood </span></td>
<td colspan="5" style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Coastal</h5>
<center> <table style="height: 130px; width: 500px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">75 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">100 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">200 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">1000 years </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Max. Depth of Flooding in the Coastal Undefended in Meters </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Coastal Undefended Flood Score </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property Modelled for Coastal Flood </span></td>
<td colspan="4" style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Surface Water</h5>
<center> <table style="height: 130px; width: 500px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">75 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">200 years </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">1000 years </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Max. Depth of Flooding in the Surface Water Undefended in Meters </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Surface Water Undefended Flood Score </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Property Modelled for Surface Water Flood </span></td>
<td colspan="4" style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
</tr>
</tbody>
</table> </center>
</html>
""" 
    return html

In [17]:
# Creating the popup labels for Airbus/NWide  loan data

def popup_html_index(row):
    i = row
    uprn_id=df_nationwide_loanbook['UPRN'].iloc[i] 
    address= ' MASKED ADDRESS '+  str( df_nationwide_loanbook['POSTCODE_FLAG'].iloc[i] ) +'         '+ str( df_nationwide_loanbook['EASTING'].iloc[i] ) +' '+ str( df_nationwide_loanbook['NORTHING'].iloc[i] ) +'         '+ str( df_nationwide_loanbook['LAT'].iloc[i] ) +' '+ str( df_nationwide_loanbook['LON'].iloc[i] )
    river_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    coastal_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    surf_water_flooding_first = df_nationwide_loanbook['RHIGHSCORE'].iloc[i]
    
    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px"> UPRN :{}</h4>""".format(uprn_id) + """</center>
<center> <table style="height: 130px; width: 600px;">
<tbody>
<tr>
<td style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Address </span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">"""+ address + """</td>
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current EPC Energy Rating and Consumption</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_CURRENT'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Potential EPC Energy Rating and Consumption</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} ({} kWh/m\u00b2/year)</td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_RATING'].iloc[i],df_nationwide_loanbook['ENERGY_CONSUMPTION_POTENTIAL'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current CO\u2082 Emissions</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_CURRENT'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Potential CO\u2082 Emissions</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} CO\u2082 Tonnes/year </td>""".format(df_nationwide_loanbook['CO2_EMISSIONS_POTENTIAL'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Energy Efficiency</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['CURRENT_ENERGY_EFFICIENCY'].iloc[i]) + """
</tr>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Potential Energy Efficiency</span></td>
<td style="border: 1px solid white; width: 150px;background-color: """+ right_col_color +""";">{} Based on cost of energy, i.e. heating, water and lighting [in kWh/year] mult. by fuel costs. (£/m²/year cost is derived from kWh). </td>""".format(df_nationwide_loanbook['POTENTIAL_ENERGY_EFFICIENCY'].iloc[i]) + """
</tr>
</tbody>
</table>
</center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">River</h5>
<center> <table style="height: 130px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style= "border: 1px solid white; width=100px; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the undefended/defended river scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['RHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Coastal</h5>
<center> <table style="height: 130px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the undefended/defended coastal scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['CHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['CHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Surface Water</h5>
<center> <table style="height: 130px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the undefended/defended surface water scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['SWHIGHSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['SWHIGHSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Ground Water</h5>
<center> <table style="height: 130px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y highest </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> Highest score from the ground water scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['GWHIGHSCORE'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">All Scores</h5>
<center> <table style="height: 130px; width: 600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"></span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y undefended </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> 2023y defended </span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;"> The sum of all the undefended/defended high scores </span></td>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['COMBINEDSCORE'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['COMBINEDSCORE_ADJ'].iloc[i]) + """
</tr>
</tbody>
</table> </center>

<h5 style="margin-bottom:5px; margin-left:5px; font-weight: bold">Loan Details</h5>
<center> <table style="height: 130px; width:600px;">
<thead>
<tr>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">UPRN </span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan Balance</span></th>
<th style= "border: 1px solid white; background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Current Loan to Value LTV</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">{}</td>""".format(df_nationwide_loanbook['UPRN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">£{}</td>""".format(df_nationwide_loanbook['CURRENTLOAN'].iloc[i]) + """
<td style="border: 1px solid white; background-color: """+ right_col_color +""";">%{}</td>""".format(df_nationwide_loanbook['CURRENTLTV'].iloc[i]) + """
</tr>
</tbody>
</table> </center>
</html>
""" 
    return html

In [18]:
#Creating lagends

def add_legend(maps, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_label = dict(zip(labels, colors))
    
    legends = ""     
    for label, color in color_label.items():
        legends += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='legend' class='legend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legends}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var Layout = (function() {{
                    var flag = false;
                    return function() {{
                        if (!flag) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!flag)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          flag = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        Layout()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .legend {
        z-index:9999;
        float:right;
        background-color: rgb(255, 255, 255);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .legend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .legend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .legend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .legend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .legend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .legend a {
        color: #777;
        }
    </style>
    """

    maps.get_root().header.add_child(folium.Element(script + css))

    return maps

In [19]:
#

# Display - NationWide & Loanbook  search by index ecad_id - UK Bristol County

In [20]:
#Plotting gamma_loan data - index search

bristol_airbus_nwide_loanbook=folium.Map(location=[lat,lang],zoom_start=10)
from folium import plugins
cluster=plugins.MarkerCluster().add_to(bristol_airbus_nwide_loanbook)

for d in df_nationwide_loanbook.iterrows(): 
        html = popup_html_index(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=600)
        folium.Marker(location=[d[1]["LAT"], d[1]["LON"]], popup=popup,name=d[1]["UPRN"]).add_to(cluster)
Search(cluster,search_label='name',placeholder='Search for ECAD ID').add_to(bristol_airbus_nwide_loanbook)            
bristol_airbus_nwide_loanbook.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Loanbook_IndexSearch_map27Apr2023.html')

# Display Optional
# bristol_airbus_nwide_loanbook

In [21]:
#Plotting flood_rating data

bristol_flood=folium.Map(location=[lat,lang],zoom_start=10)

for d in df_nationwide_loanbook.iterrows():
        html = popup_html_flood(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        folium.CircleMarker(
                    [d[1]["LAT"], d[1]["LON"]],
                    radius=6,
                    color=colors_flood[d[1]["RHIGHSCORE_ADJ"]],
                    fill=True,
                    fill_color=colors_flood[d[1]["RHIGHSCORE_ADJ"]],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(bristol_flood)

bristol_flood = add_legend(sligo_flood, 'Likelihood of flooding', colors = list(colors_flood.values()), labels = ('Very Low', 'Low', 'Moderate', 'Moderate to High', 'High', 'Very High'))
bristol_flood.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Flood_Rating.html')

# Display Optional 
#bristol_flood

KeyError: 5

In [22]:
#Plotting EPC_rating data

bristol_epc=folium.Map(location=[lat,lang],zoom_start=10)

for d in df_nationwide_loanbook.iterrows():
        html = popup_html_epc(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        folium.CircleMarker(
                    [d[1]["LAT"], d[1]["LON"]],
                    radius=6,
                    color=colors_epc[d[1]["CURRENT_ENERGY_RATING"]],
                    fill=True,
                    fill_color=colors_epc[d[1]["CURRENT_ENERGY_RATING"]],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(bristol_epc)

bristol_epc = add_legend(bristol_epc, 'Building Energy Rating', colors = list(colors_epc.values()), labels = list(colors_epc.keys()))
bristol_epc.save('data/output/visualization_Bristol_Airbus_Nationwide_loanbook/Bristol_Airbus_Nwide_Energy_Rating.html')

# Display Optional
# bristol_epc

KeyError: nan

In [23]:
exit()

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
